In [1]:
<a href="https://colab.research.google.com/github/saponew/James-Bond/blob/main/Random_Forest_Bond.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SyntaxError: invalid syntax (<ipython-input-1-197c82ab713e>, line 1)

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
from joblib import dump, load
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from collections import Counter
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.metrics import classification_report_imbalanced
from imblearn.combine import SMOTEENN
from imblearn.ensemble import BalancedRandomForestClassifier

In [ ]:
# Set path to CSV
data = Path('/content/drive/MyDrive/Project 2/Data/Data tables/all_cusips_core_rating_shifted_df_v4_MACRO_FINAL.csv')
df = pd.read_csv(data)
print(df.info)
print(df.columns)
df.head(10)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# data wrangling: drop NaN, cap timedelta @60
df_final = df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
df_final.dropna(inplace=True)
df_final = df_final.loc[df_final['timedelta'] <= 60]

print(df_final.info)
df_final.head()

In [ ]:
# Set x variable list of features
x_var_list = ['siccode_fundamentals', 'revenueusd', 'ebitdausd',
       'ebitdamargin', 'netinccmnusd', 'netmargin', 'cashnequsd',
       'workingcapital', 'debtusd', 'liabilities', 'equityusd', 'marketcap',
       'ev', 'fcf', 'ncfdebt', 'currentratio', 'de', 'divyield', 'epsusd',
       'payoutratio', 'evebitda', 'pb', 'pe', 'close_edi', 'coupon_x',
       'percent_fred_macro_yc_30_yr',
       'percent_fred_macro_yc_3_mo', 'percentchange_fred_macro_gdp',
       'dollars_fred_crude_oil_prices',
       'percent_fred_five_year_breakeven_inflation', 'index_fred_macro_vix', 'maturity_year']

# Filter by x-variable list
df_final[x_var_list].tail()

In [ ]:
X = df_final[x_var_list]
y = df_final['rank_change']

X['coupon_x'].replace('%', '', regex=True, inplace=True)
X['coupon_x'] = X['coupon_x'].astype('float') / 100

# np.unique(y, return_counts = True)
Counter(y)

In [ ]:
# convert y to array
# y = df_final['rank_diff'].values.reshape(-1, 1)

In [ ]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=78)

In [ ]:
np.unique(y_train, return_counts = True)

In [ ]:
y_test[:10]

In [ ]:
# Creating the scaler instance
scaler = StandardScaler()

# Fitting the scaler
scaler.fit(X_train)

# Transforming the data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators= (int(len(x_var_list)/3)), max_depth=3, max_samples = None, random_state=0)

# Fit the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Make a prediction of "y" values from the X_test dataset
predictions = rf_model.predict(X_test_scaled)
print(np.unique(predictions))
print(np.unique(y_test))

# Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
# Results = y_test.to_frame()
# Results["Predicted Value"] = predictions
# Results

In [ ]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 1","Actual 2", "Actual 3"], 
               columns=["Predicted 1","Predicted 2", "Predicted 3"])

# cm_df = pd.DataFrame(
#     cm, index=["Actual 1","Actual 2", "Actual 3", "Actual 4", "Actual 5", "Actual 6", "Actual 7",
#                "Actual 8", "Actual 9", "Actual 10", "Actual 11", "Actual 12", "Actual 13","Actual 14","Actual 15", "Actual 16",
#                "Actual 17", "Actual 18", "Actual 19"], 
#                columns=["Predicted 1","Predicted 2", "Predicted 3", "Predicted 4", "Predicted 5", "Predicted 6", 
#                         "Predicted 7", "Predicted 8", "Predicted 9", "Predicted 10", "Predicted 11", "Predicted 12", "Predicted 13",
#                         "Predicted 14","Predicted 15", "Predicted 16", "Predicted 17", "Predicted 18", "Predicted 19"])

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
print("Balanced accuracy score: %.4f" % balanced_accuracy_score(y_test, predictions))

In [ ]:
# Print the imbalanced classification report
print('Classification Report Imbalanced')
print(classification_report_imbalanced(y_test, predictions))

In [ ]:
# Get the feature importance array
importances = rf_model.feature_importances_

In [ ]:
# List the top 10 most important features
importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
importances_sorted[:10]

In [ ]:
# Visualize the features by importance
importances_df = pd.DataFrame(sorted(zip(rf_model.feature_importances_, X.columns), reverse=True))
importances_df.set_index(importances_df[1], inplace=True)
importances_df.drop(columns=1, inplace=True)
importances_df.rename(columns={0: 'Feature Importances'}, inplace=True)
importances_sorted = importances_df.sort_values(by='Feature Importances')
importances_sorted.plot(kind='barh', color='lightgreen', title= 'Features Importances', legend=False)

In [ ]:
importances_sorted.to_csv('/content/drive/MyDrive/Project 2/Data/Data tables/smote_rf_importance_df.csv')

In [ ]:
# RANDOM OVERSAMPLING
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

In [ ]:
# Create a random forest classifier
rf_ro_model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)

# Fit the model
rf_ro_model = rf_ro_model.fit(X_resampled, y_resampled)

# Make a prediction of "y" values from the X_test dataset
predictions = rf_ro_model.predict(X_test_scaled)

In [ ]:
# Display the confusion matrix
y_pred = rf_ro_model.predict(X_test_scaled)
confusion_matrix(y_test, y_pred)

In [ ]:
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# Get the feature importance array
importances = rf_ro_model.feature_importances_

In [ ]:
# List the top 10 most important features
importances_sorted = sorted(zip(rf_ro_model.feature_importances_, X.columns), reverse=True)
importances_sorted[:10]

In [ ]:
# Visualize the features by importance
importances_df = pd.DataFrame(sorted(zip(rf_ro_model.feature_importances_, X.columns), reverse=True))
importances_df.set_index(importances_df[1], inplace=True)
importances_df.drop(columns=1, inplace=True)
importances_df.rename(columns={0: 'Feature Importances'}, inplace=True)
importances_sorted = importances_df.sort_values(by='Feature Importances')
importances_sorted.plot(kind='barh', color='lightgreen', title= 'Features Importances', legend=False)

In [ ]:
# PRECISION RECALL CURVE

In [ ]:
from sklearn.metrics import precision_recall_curve

probs_lr = rf_model.predict_proba(X_test)[:, 1]
probs_rf = rf_ro_model.predict_proba(X_test)[:, 1]
precision_lr, recall_lr, _ = precision_recall_curve(y_test, probs_lr, pos_label=1)
precision_rf, recall_rf, _ = precision_recall_curve(y_test, probs_rf, pos_label=1)

In [ ]:
plt.plot(recall_lr, precision_lr, marker='.')
plt.plot(recall_rf, precision_rf, marker='x')

In [ ]:
# SMOTE OVERSAMPLING
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy= {-1:10000, 0:10000, 1:10000}).fit_resample(
    X_train_scaled, y_train)

Counter(y_resampled)

In [ ]:
# Create a random forest classifier
rf_model_SM = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)

# Fit the model
rf_model_SM = rf_model_SM.fit(X_resampled, y_resampled)

# Make a prediction of "y" values from the X_test dataset
predictions = rf_model_SM.predict(X_test_scaled)

In [ ]:
# Display the confusion matrix
y_pred = rf_model_SM.predict(X_test_scaled)
confusion_matrix(y_test, y_pred)

In [ ]:
# Balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
#SMOTEENN combination sampling
sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

In [ ]:
# Create a random forest classifier
rf_model_SO = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)

# Fit the model
rf_model_SO = rf_model_SO.fit(X_resampled, y_resampled)

# Make a prediction of "y" values from the X_test dataset
predictions = rf_model_SO.predict(X_test_scaled)

In [ ]:
# Display the confusion matrix
y_pred = rf_model_SO.predict(X_test_scaled)
confusion_matrix(y_test, y_pred)

In [ ]:
# Balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# BALANCED RANDOM FOREST CLASSIFIER

In [ ]:
brf = BalancedRandomForestClassifier(n_estimators=200, max_depth = 10, random_state=0)
brf.fit(X_train_scaled, y_train)

In [ ]:
y_pred_rf = brf.predict(X_test)